# Relative Binding Free Energy Lambda Sequence Generator

- The idea is a simple and *interactive* way to generate charge, vdW and bond lambdas for a relative free energy of binding calculation.
    - These values are available to copy (and note there is no enter in the long string of numbers although the text has flowed)
    - Graph output is also a sanity check.


- Comment on coding style: Umm, yeah this is a hack. Have done some awful things. Comments welcome.
- Has this been done before: Maybe there is already something out there. IDK.
- Will this look and work better in Bokeh or Plotly - probably

In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib inline
plt.style.use('seaborn')




In [2]:
@interact(total_bins=(10,80,1), charge_bins=(0,20,1),taper1_val=(0,1,0.01),taper2_val=(0,1,0.01))
def gen_sequence(total_bins=40, charge_bins=10, taper1_bins=3,  taper1_val=0.95, taper2_bins=3, taper2_val=0.99,charge_first=True):

    # total_bins is the total number of free energy bins (indexed from 1)
    # charge_bins is the total bins to use for charge perturbation
    # taper1_val is the taper value to start with e.g. 0.98
    # taper 2_val is the value to end taper 1 at and start taper 2
    # taper1_bins is number of bins or pnts to taper over e.g. 3 bins
    # taper2_bins is number of bins or pnts to taper over e.g. the last 3 bins then put in 3
    
    #inelegant yet simple parameter error resolution
    if total_bins<10: total_bins=10
    if taper1_val<0: taper1_val=0
    if taper2_val<taper1_val: taper2_val=taper1_val
    if taper2_val>1: taper2_val=1
    if taper1_val>taper2_val: taper1_val=taper2_val
    if taper1_bins<0: taper1_bins=0
    if taper2_bins<0: taper2_bins=0
        
    if charge_bins+taper1_bins+taper2_bins>total_bins: 
        return("ERROR: sum of charge bins and taper bins> total bins")
        
    x = np.arange(total_bins)
    if charge_bins==0:
        
        vdw_y_taper =  np.append(np.append(np.linspace(0.0,taper1_val, num=total_bins-taper1_bins-taper2_bins, endpoint=False),
                         np.linspace(taper1_val,taper2_val, num=taper1_bins, endpoint=False)),
                         np.linspace(taper2_val,1.0, num=taper2_bins, endpoint=True))
    else:
        charge_y = np.append(np.linspace(0.0,1.0, num=charge_bins, endpoint=True),np.linspace(1.0,1.0,num=total_bins-charge_bins, endpoint=True))
    #vdw_y =  np.append(np.linspace(0.0,0.0,num=charge_bins, endpoint=True),np.linspace(0.0,1.0, num=total_bins-charge_bins, endpoint=True))
        vdw_y_taper =  np.append(np.append(np.append(np.linspace(0.0,0.0,num=charge_bins-1, endpoint=True),np.linspace(0.0,taper1_val, num=total_bins-charge_bins-taper1_bins-taper2_bins+1, endpoint=False)),
                         np.linspace(taper1_val,taper2_val, num=taper1_bins, endpoint=False)),
                         np.linspace(taper2_val,1.0, num=taper2_bins, endpoint=True))
    
    if not charge_first:
        return("Oops, Not coded this VDW first feature yet!")
        # WRONG! tmp = charge_y
        #charge_y = vdw_y_taper
        #vdw_y_taper = tmp
#np.linspace(0.0,1.0, num=30, endpoint=True)[25]
    if charge_bins>0: plt.plot(x, charge_y, 'o')
    plt.plot(x, vdw_y_taper, '^')
    plt.show()
    print("Total bins (zero-based indexing):",total_bins-1)
    if charge_bins>0: print("Charge: ", np.array_str(charge_y, precision=2, max_line_width=10000).replace('[','').replace(']',''))
    print("VDW: ", np.array_str(vdw_y_taper, precision=2, max_line_width=10000).replace('[','').replace(']',''))
    print("Bond: ", np.array_str(vdw_y_taper, precision=2, max_line_width=10000).replace('[','').replace(']',''))

interactive(children=(IntSlider(value=40, description='total_bins', max=80, min=10), IntSlider(value=10, descr…